In [16]:
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf

input_node = 784
output_node = 10
layer1_node = 500
batch_size = 50
learning_rate_base = 0.8
learning_rate_decay = 0.99
regularization_rate = 0.0001
training_step = 10000
moving_average_decay = 0.99

def inference(input_tensor,avg_class,w1,b1,w2,b2):
    if avg_class == None:
        layer1 = tf.nn.relu(tf.matmul(input_tensor,w1)+b1)
        return tf.matmul(layer1,w2)+b2
    else:
        layer1 = tf.nn.relu(tf.matmul(input_tensor,avg_class.average(w1))+avg_class.average(b1))
        return tf.matmul(layer1,avg_class.average(w2))+avg_class.average(b2)

def train(mnist):
    x = tf.placeholder(tf.float32,[None,input_node],name='x-input')
    y_ =tf.placeholder(tf.float32,[None,output_node],name='y-output')
    w1 = tf.Variable(tf.truncated_normal([input_node,layer1_node],stddev=0.1))
    b1 = tf.Variable(tf.constant(0.1,shape=[layer1_node]))
    w2 = tf.Variable(tf.truncated_normal([layer1_node,output_node],stddev=0.1))
    b2 = tf.Variable(tf.constant(0.1,shape=[output_node]))
    y = inference(x,None,w1,b1,w2,b2)
    global_step = tf.Variable(0,trainable=False)
    variable_averages=tf.train.ExponentialMovingAverage(moving_average_decay,global_step)
    variable_averages_op=variable_averages.apply(tf.trainable_variables())
    average_y = inference(x,variable_averages,w1,b1,w2,b2)
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y,labels=tf.argmax(y_,1))
    cross_entropy_mean = tf.reduce_mean(cross_entropy)
    regularizer = tf.contrib.layers.l2_regularizer(regularization_rate)
    regularization = regularizer(w1)+regularizer(w2)
    loss = cross_entropy_mean+regularization
    learning_rate = tf.train.exponential_decay(learning_rate_base,global_step,mnist.train.num_examples/batch_size,learning_rate_decay)
    train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss,global_step=global_step)
    with tf.control_dependencies([train_step,variable_averages_op]):
        train_op=tf.no_op(name='train')
    correct_prediction = tf.equal(tf.argmax(average_y,1),tf.argmax(y_,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

    with tf.Session() as sess:
        tf.initialize_all_variables().run()
        validate_feed = {x:mnist.validation.images,y_:mnist.validation.labels}
        test_feed = {x:mnist.test.images,y_:mnist.test.labels}
        for i in range(training_step):
            if i%1000==0:
                validate_acc=sess.run(accuracy,feed_dict=validate_feed)
                print("after %d train step,validate accuracy" "using average model is %g" % (i,validate_acc))
            xs,ys=mnist.train.next_batch(batch_size)
            sess.run(train_op,feed_dict={x:xs,y_:ys})
        test_acc = sess.run(accuracy,feed_dict=test_feed)
        print("after %d train step,test accuracy using average" "model is %g" % (training_step,test_acc))

def main(argv=None):
    mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
    train(mnist)
    
if __name__=='__main__':
    tf.app.run()

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
after 0 train step,validate accuracyusing average model is 0.084
after 1000 train step,validate accuracyusing average model is 0.9764
after 2000 train step,validate accuracyusing average model is 0.9806
after 3000 train step,validate accuracyusing average model is 0.981
after 4000 train step,validate accuracyusing average model is 0.9824
after 5000 train step,validate accuracyusing average model is 0.9832
after 6000 train step,validate accuracyusing average model is 0.9836
after 7000 train step,validate accuracyusing average model is 0.9836
after 8000 train step,validate accuracyusing average model is 0.9846
after 9000 train step,validate accuracyusing average model is 0.9846
after 10000 train step,test accuracy using averagemodel is 0.9843


SystemExit: 

C:\Users\77518\AppData\Local\conda\conda\envs\tensorflow\lib\site-packages\IPython\core\interactiveshell.py:2870: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
